# Generating Venn diagrams from "youtube.py --video" outputs

## Load libraries and define functions

In [ ]:
import pandas as pd
%matplotlib inline
from matplotlib_venn import venn2, venn3
import matplotlib.pyplot as plt
import numpy as np
import math, itertools
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3
import numpy as np

# Generate list index for itertools combinations
def gen_index(n):
    x = -1
    while True:       
        while True:
            x = x + 1
            if bin(x).count('1') == n:
                break
        yield x

# Generate all combinations of intersections
def make_intersections(sets):
    l = [None] * 2**len(sets)
    for i in range(1, len(sets) + 1):
        ind = gen_index(i)
        for subset in itertools.combinations(sets, i):
            inter = set.intersection(*subset)
            l[next(ind)] = inter
    return l

# Get weird reversed binary string id for venn
def number2venn_id(x, n_fill):
    id = bin(x)[2:].zfill(n_fill)
    id = id[::-1]
    return id

# Iterate over all combinations and remove duplicates from intersections with
# more sets
def sets2dict(sets):
    l = make_intersections(sets)
    d = {}
    for i in range(1, len(l)):
        d[number2venn_id(i, len(sets))] = l[i]
        for j in range(1, len(l)):
            if bin(j).count('1') < bin(i).count('1'):
                l[j] = l[j] - l[i]
                d[number2venn_id(j, len(sets))] = l[j] - l[i]
    return d

## Load data (the youtube.py --video csv output specified as path)

In [ ]:
path = '/home/ubuntu/dashboard/outputs/video5.csv'

df = pd.read_csv(path)

title = df['sourceTitle'][0]
df = df[['watcher','id','related_source','related_videoId','related_title','related_index']]

print(title)

## Clean the data

#### First, specify three users  among the unique ones you have in your dataset

In [ ]:
df.watcher.unique()

In [ ]:
user1, user2, user3 = df.watcher.unique()

df1 = df[df.watcher == user1]
df2 = df[df.watcher == user2]
df3 = df[df.watcher == user3]

#### Then, you need to pick just one session (id) per user.

In [ ]:
df1.id.unique()

In [ ]:
df2.id.unique()

In [ ]:
df3.id.unique()

In [ ]:
df1 = df1[df1.id == '47c16bdc2358e51110003a626d822a61e6bc908f']
df2 = df2[df2.id == '120bce0960e72e333ebc5f95e48611c6b8d64b20']
df3 = df3[df3.id == 'b2a48bdfddc7a9a0cd0f706da9c850dc790d7c41']

#### Then we create the three clean sets with one id each

In [ ]:
A = set(df1.related_source)
B = set(df2.related_source)
C = set(df3.related_source)

sets_source = [A, B, C]

## Then we apply our function and visualize the venn diagram

### Source name

In [ ]:
d = sets2dict(sets_source)

# Plot it
plt.figure(figsize=(40,20))
h = venn3(sets_source, (user1, user2, user3))
for k, v in d.items():
   l = h.get_label_by_id(k)
   if l:
       l.set_fontsize(12)
       l.set_text('\n'.join(sorted(v)))
